In [2]:
import numpy as np
import pandas as pd
import xlsxwriter

In [4]:
db = pd.read_excel("../DataFiles/CompleteWeatherHist2000-2013.xlsx")

In [5]:
db.head(10)

,Airport,Year,Month,Day,Max TemperatureF,Mean TemperatureF,Min TemperatureF,Max Dew PointF,MeanDew PointF,Min DewpointF,...,Max VisibilityMiles,Mean VisibilityMiles,Min VisibilityMiles,Max Wind SpeedMPH,Mean Wind SpeedMPH,Max Gust SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
0,KBFI,2000,11,12,45,40,34,35,32,28,...,10,10,8,9,5,NaN,0.00,8,NaN,143
1,NaN,2000,6,9,55,NaN,55,47,47,47,...,10,10,10,12,12,16,0.00,6,NaN,220
2,NaN,2000,12,5,50,42,33,38,36,34,...,10,10,10,7,3,NaN,0.00,3,NaN,359
3,NaN,2000,1,3,46,40,35,42,37,32,...,10,7,2,15,7,NaN,0.32,8,Rain,143
4,NaN,2000,1,2,43,40,36,35,32,29,...,10,10,10,12,8,NaN,0.00,6,NaN,177
5,NaN,2000,1,1,44,42,39,37,30,28,...,10,9,4,20,11,30,0.27,8,Rain,175
6,NaN,2000,1,7,46,44,41,45,41,31,...,10,8,3,15,8,NaN,0.29,8,Rain,161
7,NaN,2000,1,6,42,37,32,39,32,30,...,10,9,4,12,6,NaN,0.02,5,Rain,147
8,NaN,2000,1,5,44,40,35,38,36,32,...,10,9,6,8,3,NaN,0.03,6,Rain,134
9,NaN,2000,1,4,48,45,41,45,40,37,...,10,9,3,21,11,30,0.25,6,Rain,192


In [6]:
db.loc[:,"ApVal"] = pd.isnull(db.loc[:,"Airport"])  # boolean type indicating whether the airport is null.

In [7]:
# The excel file format had merged all the rows that belonged to the "airport code" column and when imported into
# pandas, the merged rows are split with the contents for the cells allocated to the first cell, leaving the rest
# of the cells blank, the following code allocates code-names respectively to restore the intended assignments
for i in range(len(db)):
    if db.loc[i,"ApVal"]:
        db.loc[i, "Airport"] = db.loc[(i-1), "Airport"]

In [42]:
# db.PrecipitationIn.astype(float)

In [39]:
db.PrecipitationIn.dtype

dtype('O')

In [ ]:
# sample.tail(30)

In [ ]:
# sample.columns

In [8]:
codes = ['KBFI', 'KPUW', 'KPSC', 'KESW', 'KALW', 'KTDO', 'KVUO', 'KPWT', 'KMWH', 'KPAE', 'KEPH', 
         'KSEA', 'KHQM', 'KUIL', 'KGRF', 'KTIW', 'KSKA', 'KEAT', 'KOLM', 'KTCM', 'KYKM', 'KBLI', 'KGEG', 
         'KAWO', 'KSFF', 'KCLM', 'KDLS', 'KRLD']

In [51]:
monthlyR = {}
for ap in codes:
    monthlyR[ap] = {}
    for y in np.arange(14)+2000:
        for m in np.arange(12)+1:
            sampleDB = db[(db.loc[:, "Airport"] == ap) & ((db.loc[:, "Year"] == y) & (db.loc[:, "Month"] == m))]
#             summaryT = sampleDB[['Max TemperatureF','Min TemperatureF']].mean(0)
#             maxT = summaryT[0]
#             minT = summaryT[1]
#             prep = sampleDB[['PrecipitationIn']].sum(0)[0]
            ftCycles = len(sampleDB[((((sampleDB.loc[:,'Events'] == "Snow") | ((sampleDB.loc[:,'Events'] == "Rain-Snow")|
                                       (sampleDB.loc[:,'Events'] == "Rain-Snow-Thunderstorm"))) |
                                   ((sampleDB.loc[:,'Events'] == "Fog-Rain-Snow") | (sampleDB.loc[:,'Events'] == "Fog-Rain-Snow-Hail"))) |
                                   ((sampleDB.loc[:,'Events'] == "Fog-Rain-Snow-Hail-Thunderstorm") | 
                                    (sampleDB.loc[:,'Events'] == "Fog-Rain-Snow-Thunderstorm"))) |
                                   (((sampleDB.loc[:,'Events'] == "Fog-Snow") | 
                                    (sampleDB.loc[:,'Events'] == "Fog-Snow-Thunderstorm")) |
                                   (((sampleDB.loc[:,'Events'] == "Fog-Snow-Hail") | 
                                    (sampleDB.loc[:,'Events'] == "Fog-Snow-Hail-Thunderstorm")) |
                                   ((sampleDB.loc[:,'Events'] == "Rain-Snow-Hail") | 
                                    (sampleDB.loc[:,'Events'] == "Rain-Snow-Hail-Thunderstorm"))))])
            vals = ftCycles
            key  = str(y)+'-'+ str(m)
            monthlyR[ap][key] = vals

In [52]:
frames = []
for key in monthlyR.keys():
    pdb = pd.DataFrame.from_dict(monthlyR[key], orient = 'index')
    pdb['Name'] = key
    frames.append(pdb)
wdataM = pd.concat(frames)

In [53]:
len(wdataM) == 28*240

False

In [54]:
wdataM.head(20)

,0,Name
2003-12,3,KBFI
2004-6,0,KBFI
2003-10,0,KBFI
2003-11,2,KBFI
2004-3,0,KBFI
2004-2,0,KBFI
2004-1,4,KBFI
2007-4,0,KBFI
2007-5,0,KBFI
2007-6,0,KBFI


In [55]:
wdataM.to_excel("../DataFiles/monthlyWeatherR2.xlsx")

In [22]:
# In the following lines, I wanted to check whether the precipitation variable had no erroneous values. However, got 
# a better dataset so this one nolonger mattered that much. I reviewed this code to extract the number of freeze thaw
# cycles for the new weather dataset
# tdb = db[(db.loc[:, "Airport"] == 'KDLS') & ((db.loc[:, "Year"] == 2005))]
# tdb[tdb.loc[:,"PrecipitationIn"] > 1000.0]

In [60]:
db2 = pd.read_excel("../DataFiles/monthlyWeatherR2.xlsx")

In [61]:
AnnualR = {}
for ap in codes:
    AnnualR[ap] = {}
    for y in np.arange(14)+2000:
        sampleDB = db2[(db2.loc[:, "Name"] == ap) & (db2.loc[:, "Year"] == y)]
#         summaryT = sampleDB[['MaxT','MinT']].mean(0)
#         AMmaxT = sampleDB[['MaxT']].max(0)[0]
#         AMminT = sampleDB[['MinT']].min(0)[0]
#         AAmaxT = summaryT[0]
#         AAminT = summaryT[1]
#         AnnualPrecFT = sampleDB[['Precipitation', 'FTCycles']].sum(0)
#         Aprep = AnnualPrecFT[0]
        AftCycles = sampleDB[['FTCycles']].sum(0)
        vals = AftCycles
        key  = str(y)
        AnnualR[ap][key] = vals

In [62]:
len(AnnualR.keys())

28

In [63]:
test = pd.DataFrame.from_dict(AnnualR['KBFI'], orient = 'index')

In [64]:
test.head()

,FTCycles
2000,3
2001,5
2002,9
2003,5
2004,4


In [65]:
frames2 = []
for key in AnnualR.keys():
    pdb = pd.DataFrame.from_dict(AnnualR[key], orient = 'index')
    pdb['Airport'] = key
    frames2.append(pdb)
wdataAnnual = pd.concat(frames2)

In [66]:
wdataAnnual.head()

,FTCycles,Airport
2000,3,KBFI
2001,5,KBFI
2002,9,KBFI
2003,5,KBFI
2004,4,KBFI


In [67]:
wdataAnnual.columns = ["FTCycles", "Airport" ]

In [68]:
wdataAnnual.to_excel("../DataFiles/AnnualWeatherR2.xlsx")